In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

bitcoin = pd.read_csv('Bitcoin2.csv', sep=';')
bitcoin = bitcoin.iloc[::-1].reset_index(drop=True)

In [19]:
diff_bitcoin = np.diff(bitcoin['close'].values, n=1)
df_diff = pd.DataFrame({'diff_bitcoin': diff_bitcoin})

In [22]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def rolling_forecast(df: pd.DataFrame, train_len: int, horizon: int, window: int, method: str) -> list:
    total_len = train_len + horizon
    end_idx = train_len
    if method == 'mean':
        pred_mean = []
        for i in range(train_len, total_len, window):
            mean = np.mean(df[:i].values)
            pred_mean.extend(mean for _ in range(window))
        return pred_mean
    elif method == 'last':
        pred_last_value = []
        for i in range(train_len, total_len, window):
            last_value = df[:i].iloc[-1].values[0]
            pred_last_value.extend(last_value for _ in range(window))
        return pred_last_value
    elif method == 'ARMA':
        pred_ARMA = []
        for i in range(train_len, total_len, window):
            model = SARIMAX(df[:i], order=(2,0,5))
            res = model.fit(disp=False)
            predictions = res.get_prediction(0, i + window - 1)
            oos_pred = predictions.predicted_mean.iloc[-window:]
            pred_ARMA.extend(oos_pred)
        return pred_ARMA

In [23]:
TRAIN_LEN = len(df_diff)
HORIZON = 5
WINDOW = 5

pred_mean = rolling_forecast(df_diff, TRAIN_LEN, HORIZON, WINDOW, 'mean')
pred_last_value = rolling_forecast(df_diff, TRAIN_LEN, HORIZON, WINDOW, 'last')
pred_ARMA = rolling_forecast(df_diff, TRAIN_LEN, HORIZON, WINDOW, 'ARMA')

In [24]:
df_pred = pd.DataFrame({'pred_ARMA': pred_ARMA})
print('Current Price ($) = ', bitcoin['close'].iloc[363])
bitcoin['close'].iloc[363] + df_pred['pred_ARMA'].cumsum()

Current Price ($) =  64031.1328085695


0    65328.159357
1    65140.120333
2    64098.491768
3    64643.794719
4    65496.788475
Name: pred_ARMA, dtype: float64